# 1. Import and Install Dependencies

In [1]:
!pip3 install tensorflow==2.13.0 opencv-python mediapipe scikit-learn matplotlib pygame

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pygame

2023-12-23 20:51:11.924597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['L'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    #for sequence in range(1,no_sequences+1):
     for sequence in range(no_sequences):
        try: 
            #os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing (SKIP for realtime prediction)

In [10]:
import time
pygame.init()
pygame.mixer.init()
ready_sound=pygame.mixer.Sound("getready.wav")
start_sound=pygame.mixer.Sound("startpose.wav")
end_sound=pygame.mixer.Sound("endpose.wav")
pose_achieved=pygame.mixer.Sound("poseachieved.wav")

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.9) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            ready_sound.play()
            #cv2.waitKey(3000)
            time.sleep(3)
            start_sound.play()
            #cv2.waitKey(1000)
            time.sleep(1)
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)   
                    cv2.putText(image, 'STARTING', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            end_sound.play()
            #cv2.waitKey(3000)
            time.sleep(3)
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1703291965.906076       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-22.1.28), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-12-22 19:39:30.141 Python[5147:95394] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [10]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

# 6. Preprocess Data and Create Labels and Features (SKIP for realtime prediction)

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'L': 0}

In [13]:
sequences, labels = [], []
for action in actions:
    lst = os.listdir(os.path.join(DATA_PATH, action))
    if '.DS_Store' in lst:
        lst.remove('.DS_Store')
    for sequence in np.array(lst).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
# Ok to skip
np.array(sequences).shape

(30, 30, 1662)

In [15]:
# Ok to skip
np.array(labels).shape

(30,)

In [16]:
# Ok to skip
X = np.array(sequences)

In [17]:
# Ok to skip
X.shape

(30, 30, 1662)

In [18]:
# Ok to skip
y = to_categorical(labels).astype(int)

In [19]:
# Ok to skip
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
# Ok to skip
y_test.shape

(2, 1)

# 7. Build and Train LSTM Neural Network (SKIP for realtime prediction)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [32]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='relu'))

In [33]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])

In [34]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 2.0850 - categorical_accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 73ms/step - loss: 15.4249 - categorical_accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 75ms/step - loss: 15.4249 - categorical_accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 77ms/step - loss: 15.4249 - categorical_accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 66ms/step - loss: 15.4249 - categorical_accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 56ms/step - loss: 15.4249 - categorical_accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 81ms/step - loss: 14.8741 - categorical_accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 52ms/step - loss: 14.8741 - categorical_accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 66ms/step - loss: 14.3232 - categorical_accuracy: 1.0000
Epoc

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 479361 (1.83 MB)
Trainable params: 479361 (1.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 8. Make Predictions (SKIP for realtime prediction)

In [36]:
res = model.predict(X_test)

1/1 [==============================] - 0s 383ms/step


In [37]:
res

array([[3954.2783],
       [4067.9824]], dtype=float32)

In [38]:
if res[0]/4000 > 0.9 :
    print(action[0])

L


# 9. Save Weights (SKIP for realtime prediction)

In [39]:
model.save('action.keras')

In [40]:
del model

# 10. Test in Real Time

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [41]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='relu'))

In [42]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])

In [43]:
model.load_weights('action.keras')

# 11. Prediction

In [44]:
from scipy import stats

In [45]:
# 1. New detection variables
sequence = []
threshold = 0.9

pygame.init()
pygame.mixer.init()
ready_sound=pygame.mixer.Sound("getready.wav")
start_sound=pygame.mixer.Sound("startpose.wav")
end_sound=pygame.mixer.Sound("endpose.wav")
pose_achieved=pygame.mixer.Sound("poseachieved.wav")


cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.9) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            if res/4000 > threshold:
                pose_achieved.play()
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(30000)
                break
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1703383050.216847       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-22.1.28), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1/1 [==============================] - 0s 440ms/step
[0.0945651]
1/1 [==============================] - 0s 30ms/step
[0.0945651]
1/1 [==============================] - 0s 27ms/step
[0.0945651]
1/1 [==============================] - 0s 28ms/step
[0.0945651]
1/1 [==============================] - 0s 22ms/step
[0.0945651]
1/1 [==============================] - 0s 22ms/step
[0.0945651]
1/1 [==============================] - 0s 23ms/step
[0.0945651]
1/1 [==============================] - 0s 23ms/step
[0.0945651]
1/1 [==============================] - 0s 23ms/step
[0.0945651]
1/1 [==============================] - 0s 23ms/step
[0.0945651]
1/1 [==============================] - 0s 23ms/step
[0.0945651]
1/1 [==============================] - 0s 22ms/step
[0.0945651]
1/1 [==============================] - 0s 24ms/step
[0.0945651]
1/1 [==============================] - 0s 23ms/step
[0.0945651]
1/1 [==============================] - 0s 22ms/step
[0.0945651]
1/1 [==============================] - 

2023-12-23 20:57:58.390 Python[2689:36868] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
